In [9]:
import pandas as pd 
import numpy as np 

from sklearn import preprocessing
from sklearn import metrics, cross_validation
from sklearn.cluster import KMeans

from sklearn.utils import resample

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.multiclass import OneVsRestClassifier

from sklearn.grid_search import GridSearchCV

import matplotlib.pyplot as plt 
import seaborn as sns

In [10]:
df= pd.read_csv("Training_Dataset.csv")

In [11]:
df.head()

,cm_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,mvar48,mvar49,mvar50,mvar51
0,1,Charge,0,106920,4,18,970,0.7969,0.1831,337236,...,0,0,0,0,0,0,1,0,0,0
1,2,Charge,2,0,4,19,1030,0.0678,0.0000,2918974,...,0,0,0,0,1,0,0,0,0,0
2,3,Charge,2,399168,4,35,327,0.7551,0.2986,498656,...,0,0,0,0,0,0,1,0,0,0
3,4,Charge,0,267300,4,274,1258,0.7969,0.5297,2480074,...,0,1,0,0,0,0,1,0,0,1
4,5,Charge,0,213840,6,1,0,2.3238,0.0000,2918974,...,0,0,0,0,1,0,0,1,0,0


In [12]:
spend = df.mvar36 + df.mvar37 + df.mvar38 + df.mvar39

In [13]:
income = df["mvar9"][df.mvar9>0]


In [14]:
income= income.sort_values(ascending=True).reset_index()

In [15]:
median_income= income.median()[1]

In [16]:
df.mvar9.replace(to_replace=0, value=median_income, inplace=True)

In [17]:
le= preprocessing.LabelEncoder()
df.mvar12= le.fit_transform(df.mvar12)

In [18]:
#df2a= df2.ix[:, 0:13]

In [19]:
#df2b= df2.iloc[:,40:43]
#df2b

In [20]:
#df2= pd.concat([df2a, df2b], axis=1)
#df2.head()

In [21]:
df["spend"]=spend

In [22]:
#df["supplementary_offered"]= df.mvar46

In [23]:
#df2["supplementary"]= df.mvar49

In [24]:
df.mvar9 = np.log(df.mvar9)

In [25]:
df.spend= (df.spend- df.spend.mean())/df.spend.std()

In [26]:
df.mvar6 = (df.mvar6- df.mvar6.mean())/df.mvar6.std()

In [27]:
df.mvar50 = df.mvar50*2
df.mvar51 = df.mvar51*3

In [28]:
df["card_accepted"]= df.mvar49 +df.mvar50 + df.mvar51

In [29]:
df.drop(["mvar51", "mvar50", "mvar49", "mvar48", "mvar47","mvar46",], axis=1, inplace= True)

In [30]:
df2=df

In [31]:
df2= df2.iloc[:, 0:16]

In [32]:
df2= pd.concat([df2, df.iloc[:, 43:]], axis=1)

In [33]:
df2.drop(["cm_key", "mvar1", "mvar3"], axis=1, inplace= True)

In [34]:
df2_majority = df2[df2.card_accepted==0]
df2_supp = df2[df2.card_accepted==1]
df2_elite = df2[df2.card_accepted==2]
df2_credit = df2[df2.card_accepted==3]

In [35]:
print df2_majority.shape
print (df2_supp.shape[0])
print df2_elite.shape[0]
print df2_credit.shape[0]
print float(df2_supp.shape[0])/float(df2_supp.shape[0]+df2_elite.shape[0]+df2_credit.shape[0])
print df2_elite.shape[0]/float(df2_supp.shape[0]+df2_elite.shape[0]+df2_credit.shape[0])
print df2_credit.shape[0]/float(df2_supp.shape[0]+df2_elite.shape[0]+df2_credit.shape[0])


(31122, 18)
3447
2654
2777
0.388263122325
0.298941202974
0.312795674702


In [45]:
cluster_range = range( 1, 20 )
cluster_errors = []

for num_clusters in cluster_range:
    clusters = KMeans( num_clusters )
    clusters.fit( df2_majority.iloc[:,:-1] )
    cluster_errors.append( clusters.inertia_ )
clusters_df = pd.DataFrame( { "num_clusters":cluster_range, "cluster_errors": cluster_errors } )
clusters_df.head()


,cluster_errors,num_clusters
0,2.785724e+08,1
1,6.704331e+07,2
2,3.320594e+07,3
3,1.788156e+07,4
4,1.261423e+07,5


In [36]:
df2_majority_downsampled = resample(df2_majority,
                                    replace=False,
                                    n_samples=20000,
                                    random_state=123)

In [37]:
df2_supp_upsampled = resample(df2_supp,
                              replace=True,
                              n_samples=12000,
                              random_state=123)

In [38]:
df2_elite_upsampled = resample(df2_elite,
                               replace=True,
                               n_samples=10000,
                               random_state=123)

In [39]:
df2_credit_upsampled = resample(df2_credit,
                                replace=True,    
                                n_samples=10000,     
                                random_state=123)

In [40]:
df5 = pd.concat([df2_majority_downsampled, df2_supp_upsampled, df2_elite_upsampled, df2_credit_upsampled])
df5.shape

(52000, 18)

In [275]:
train, test= train_test_split(df5, test_size=0.3)

In [276]:
x_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]
x_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

In [277]:
xgb = XGBClassifier(
    n_estimators=300,
    max_depth=10, 
    min_child_weight=1,
    objective= 'multi:softmax',
 )

In [278]:
xgb.fit(x_train, y_train)

XGBClassifier(base_score=0.5, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=10, min_child_weight=1, missing=None,
       n_estimators=300, nthread=-1, objective='multi:softprob', seed=0,
       silent=True, subsample=1)

In [279]:
y_predicted = xgb.predict(x_test)

In [280]:
print(metrics.accuracy_score(y_test, y_predicted))
print(metrics.confusion_matrix(y_test, y_predicted))
print(metrics.classification_report(y_test, y_predicted))

0.88641025641
[[4961  335  426  287]
 [ 291 3309   23   19]
 [ 174    5 2763   12]
 [ 172    5   23 2795]]
             precision    recall  f1-score   support

          0       0.89      0.83      0.85      6009
          1       0.91      0.91      0.91      3642
          2       0.85      0.94      0.89      2954
          3       0.90      0.93      0.92      2995

avg / total       0.89      0.89      0.89     15600



In [227]:
test_df= pd.read_csv("Leaderboard_Dataset.csv")

In [228]:
test_income = test_df["mvar9"][test_df.mvar9>0]
test_income= test_income.sort_values(ascending=True).reset_index()
median_income= test_income.median()[1]
test_df.mvar9.replace(to_replace=0, value=median_income, inplace=True)
test_spend = test_df.mvar36 + test_df.mvar37 + test_df.mvar38 + test_df.mvar39

In [229]:
test_df.drop(["cm_key", "mvar1", "mvar3"], axis=1, inplace=True)

In [230]:
test_dfa= test_df.ix[:,0:13]

/home/apoorva/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.


In [231]:
test_dfb= test_df.ix[:, 40:46]

In [232]:
test_df= pd.concat([test_dfa, test_dfb], axis=1)

In [233]:
le = preprocessing.LabelEncoder()
test_df.mvar12 = le.fit_transform(test_df.mvar12)

In [234]:
test_df["mvar9"]= np.log(test_df["mvar9"])

In [235]:
test_df["spend"]=test_spend
test_df.spend= (test_df.spend- test_df.spend.mean())/test_df.spend.std()

In [236]:
test_df.mvar6= (test_df.mvar6- test_df.mvar6.mean())/test_df.mvar6.std()

In [281]:
test_result= xgb.predict(test_df)

In [282]:
test_df2 = pd.read_csv("Leaderboard_Dataset.csv")

In [283]:
result= pd.DataFrame(
{
    "card_accepted": test_result,
    "cm_key": test_df2.cm_key
    
})

In [284]:
result.shape

(10000, 2)

In [285]:
result.card_accepted.replace(to_replace=0, value="No_card", inplace=True)

In [286]:
result.card_accepted.replace(to_replace=1, value="Supp", inplace=True)
result.card_accepted.replace(to_replace=2, value="Elite", inplace=True)
result.card_accepted.replace(to_replace=3, value="Credit", inplace=True)

In [287]:
submission= result[result.card_accepted != "No_card"]

In [288]:
submission.shape

(1013, 2)

In [289]:
submission= submission.reindex_axis(sorted(submission.columns, reverse=True), axis=1)

In [290]:
submission= submission[:1000]

In [291]:
submission.to_csv("Reckless_Serenade_IITRoorkee_8.csv", header=False, index=False)